In [45]:
!pip install openai

In [16]:
from openai import OpenAI
client = OpenAI(api_key = api)

In [20]:
# Upload the file
uploaded_file = client.files.create(
    file=open("/content/story.txt", "rb"),
    purpose="assistants"
)

# Create a vector store and add the file
vector_store = client.beta.vector_stores.create(
    name="Story Vector Store",
    file_ids=[uploaded_file.id]
)

# Create the assistant with file_search tool
assistant = client.beta.assistants.create(
    name="Story Helper",
    description="You are a motivator who answers questions based on the story.",
    model="gpt-4o",
    tools=[{"type": "file_search"}],
    tool_resources={
        "file_search": {
            "vector_store_ids": [vector_store.id]
        }
    }
)

In [21]:
thread = client.beta.threads.create()
thread


Thread(id='thread_t7liqCxLZTM5UZwNzRfLW9OS', created_at=1739090648, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [31]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the educational qualification of the engineer and what is his name?"
)


In [32]:
message

Message(id='msg_ZOzyD8HMiARaQTLLouobecBM', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the educational qualification of the engineer and what is his name?'), type='text')], created_at=1739091439, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_t7liqCxLZTM5UZwNzRfLW9OS')

In [33]:
assistant

Assistant(id='asst_AqxXPCqgIAcUspW5G7ljyFHJ', created_at=1739090621, description='You are a motivator who answers questions based on the story.', instructions=None, metadata={}, model='gpt-4o', name='Story Helper', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_67a86abd0bc48191943979cc842d195d'])), top_p=1.0, reasoning_effort=None)

In [34]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)


In [35]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [36]:
run.status

'queued'

In [37]:
message

Message(id='msg_ZOzyD8HMiARaQTLLouobecBM', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the educational qualification of the engineer and what is his name?'), type='text')], created_at=1739091439, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_t7liqCxLZTM5UZwNzRfLW9OS')

In [38]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
messages


SyncCursorPage[Message](data=[Message(id='msg_ZOzyD8HMiARaQTLLouobecBM', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the educational qualification of the engineer and what is his name?'), type='text')], created_at=1739091439, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_t7liqCxLZTM5UZwNzRfLW9OS'), Message(id='msg_tklAFkCWhOSUCN06I6gScD7M', assistant_id='asst_AqxXPCqgIAcUspW5G7ljyFHJ', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=258, file_citation=FileCitation(file_id='file-VtDEd6UpWjqpymRKexn2dj'), start_index=243, text='【4:0†story.txt】', type='file_citation')], value='The hero of the story is Rahul, a young and ambitious engineer who works tirelessly to design an energy-efficient engine. His perseverance, teamwork, and innovation lead to a succ

In [39]:
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run.status=="completed":
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        latest_message = messages.data[0]
        text = latest_message.content[0].text.value
        print(text)
        break;


The engineer's name is Rahul, and he earned a degree in mechanical engineering【8:0†story.txt】.
